In [15]:
import os 
import pandas as pd 
import numpy as np
import json 
import math 
import re 
import requests 
import datetime
import sqlite3
from collections import deque 
from contextlib import contextmanager
from datetime import date, datetime as dt, timezone

import psycopg2
import psycopg2.extras
from bs4 import BeautifulSoup
from pprint import PrettyPrinter
from web3 import Web3
from etherscan import Etherscan
from dotenv import load_dotenv 
load_dotenv()

pprint = PrettyPrinter(indent=4)

In [16]:
# file = "documents/unfiled/Assets/US/MetaMaskHotWallet/export-0xe249d1bE97f4A716CDE0D7C5B6b682F491621C41.csv"
# df = pd.read_csv(file)
# df["UnixTimestamp"] = pd.to_datetime(df["UnixTimestamp"])
# df.head()

In [3]:
API_KEY = os.environ["ETHERSCAN_API_KEY"]
addr = os.environ["MM_HOT_ADDRESS"]
w3 = Web3(Web3.HTTPProvider(f'https://mainnet.infura.io/v3/{os.environ["INFURA_PROJECT_ID"]}'))
eth = Etherscan(API_KEY, net="MAIN") 

In [4]:
wei = eth.get_eth_balance(address=addr)
w3.fromWei(int(wei), 'ether')

Decimal('0.053137067722099653')

In [14]:
UNISWAP_V2_ROUTER = "0x7a250d5630B4cF539739dF2C5dAcb4c659F2488D"
OLYMPUS_DAO_STAKING_HELPER = "0xC8C436271f9A6F10a5B80c8b8eD7D0E8f37a612d"
OHM_TOKEN_CONTRACT = "0x383518188C0C6d7730D91b2c03a03C837814a899"
OHM_TREASURY_V2 = '0x31F8Cc382c9898b273eff4e0b7626a6987C846E8'
OHM_STAKING_V2 = "0xFd31c7d00Ca47653c6Ce64Af53c1571f9C36566a"
OHM_STAKING_V2_DEPLOYMENT_TX = "0x1a1a3dd33879ff1b765cc5fae84102990a3e866d1285e0d81d79cbff836f56e6"
OHM_STAKING_DISTRIBUTOR_V4 = '0xc58e923bf8a00e4361fe3f4275226a543d7d3ce6'

In [6]:
def get_etherscan_contract_addr_name_map(addr_wallet, flat=True): 
    """ For all transactions for a given account, we scrape mappings of smart contract addresses to human readable names 
        this data is not available in the etherscan downloads so it can be useful metadata 
        TODO: add request pagination as the number of txs for a wallet grows 
    """
    # fake a User-Agent as this bypasses captcha requirements 
    headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
    type_url_map = {
        "normal": f"https://etherscan.io/address/{addr_wallet}", 
        "erc20": f"https://etherscan.io/address-tokenpage?m=normal&a={addr_wallet}"
    }
    result = dict()
    for t, url in type_url_map.items(): 
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.content, 'html.parser')
        tables = [
            t for t in soup.find_all('table') 
            if "Txn Hash" in str(t) and re.sub("[\s\n\r]", "", t.find("tbody").getText())
        ]
        assert len(tables) == 1
        anchors = tables[0].find_all("a")
        strings = []
        for a in anchors: 
            if "title" in a.attrs:
                strings.append(a.attrs["title"])
        strings = np.unique(strings)
        addr_name_map = dict()
        for s in strings: 
            match = re.match("(.*)\n\((0x.*)\)", s)
            if match: 
                addr_name_map[match.group(2)] = match.group(1)
        result[t] = addr_name_map
    # optionally flatten the result 
    if flat: 
        result_flat = dict()
        for _, v in result.items():
            result_flat = {**result_flat, **v}
        return result_flat
    else: 
        return result

In [7]:
def date_to_datetime(d): 
    return dt(*d.timetuple()[:6], tzinfo=timezone.utc)

def utc_timestamp(dto): 
    if isinstance(dto, date): 
        return date_to_datetime(dto).replace(tzinfo=timezone.utc).timestamp()
    elif isinstance(dto, datetime): 
        return dto.replace(tzinfo=timezone.utc).timestamp()
    raise Exception("Expected input of either date or datetime")

def get_current_block_number(): 
    # get the most recent verified block number 
    now_plus_min = dt.now(tz=timezone.utc) + datetime.timedelta(minutes=1)
    timestamp = math.ceil(utc_timestamp(now_plus_min)) # seconds
    return int(eth.get_block_number_by_timestamp(timestamp, "before"))

def get_first_block_for_day(day: date): 
    # get the first block mined for a given day. handles case where first block is mined on exactly midnight 
    timestamp = int(utc_timestamp(date_to_datetime(day)))
    block = int(eth.get_block_number_by_timestamp(timestamp, "after"))
    block_data = eth.get_block_reward_by_block_number(block)
    true_datetime = dt.fromtimestamp(int(block_data["timeStamp"]), tz=timezone.utc)
    if true_datetime < date_to_datetime(day): 
        return block + 1
    else: 
        return block 
    
def get_last_block_for_day(day: date): 
    # get the last block mined for a given day
    timestamp = int(utc_timestamp((date_to_datetime(day) + datetime.timedelta(days=1))))
    block = int(eth.get_block_number_by_timestamp(timestamp, "before"))
    return block 

def get_block_range_for_day(day: dt.date): 
    return get_first_block_for_day(day), get_last_block_for_day(day)

def get_OHM_staking_block_range(): 
    """ Returns the full block range from when the staking contract was deployed until current moment in time """
    staking_deploy_tx = w3.eth.get_transaction(OHM_STAKING_V2_DEPLOYMENT_TX)
    staking_deploy_block = int(staking_deploy_tx["blockNumber"])
    cur_block = get_current_block_number()
    return staking_deploy_block, cur_block

d = dt.now().date() - datetime.timedelta(days=3)
bs = get_first_block_for_day(d)
be = get_last_block_for_day(d)
print(d)
print(bs, be)

2021-12-10
13774075 13780501


In [8]:
def get_connection(): 
    conn = psycopg2.connect("dbname=olympus user=postgres password=a")
    return conn 

def create_table_OHM_stake_address(conn, cur): 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS OHM_stake_address (
        contract_address bytea NOT NULL, 
        evt_block_number int8 NOT NULL, 
        evt_block_time timestamptz NOT NULL, 
        evt_index int8 NOT NULL, 
        evt_tx_hash bytea NOT NULL, 
        from_addr bytea NOT NULL, 
        to_addr bytea NOT NULL, 
        value numeric NOT NULL, 
        PRIMARY KEY (evt_tx_hash, from_addr, to_addr, value)
    );
    """)
    conn.commit()
            
def truncate_table_OHM_stake_address(conn, cur): 
    cur.execute("TRUNCATE TABLE OHM_stake_address;")
    conn.commit()
            
def insert_many_table_OHM_stake_address(conn, cur, records, kwargs=dict(template=None, page_size=100)): 
    insert_query = """
        INSERT INTO OHM_stake_address (
            contract_address, 
            evt_block_number, 
            evt_block_time, 
            evt_index, 
            evt_tx_hash, 
            from_addr, 
            to_addr, 
            value
        ) VALUES %s
        ON CONFLICT (evt_tx_hash, from_addr, to_addr, value) DO NOTHING
    """
    psycopg2.extras.execute_values(cur, insert_query, records, **kwargs)
    conn.commit()   

def create_table_OHM_stake_tx(conn, cur): 
    cur.execute("""
    CREATE TABLE IF NOT EXISTS OHM_stake_tx (
        contract_address bytea NOT NULL, 
        evt_block_number int8 NOT NULL, 
        evt_block_time timestamptz NOT NULL, 
        evt_index int8 NOT NULL, 
        evt_tx_hash bytea NOT NULL, 
        from_addr bytea NOT NULL, 
        to_addr bytea NOT NULL, 
        value numeric NOT NULL, 
        PRIMARY KEY (evt_tx_hash, from_addr, to_addr, value)
    );
    """)
    conn.commit()
            
def truncate_table_OHM_stake_tx(conn, cur): 
    cur.execute("TRUNCATE TABLE OHM_stake_tx;")
    conn.commit()
            
def insert_many_table_OHM_stake_tx(conn, cur, records, kwargs=dict(template=None, page_size=100)): 
    insert_query = """
        INSERT INTO OHM_stake_tx (
            contract_address, 
            evt_block_number, 
            evt_block_time, 
            evt_index, 
            evt_tx_hash, 
            from_addr, 
            to_addr, 
            value
        ) VALUES %s
        ON CONFLICT (evt_tx_hash, from_addr, to_addr, value) DO NOTHING
    """
    psycopg2.extras.execute_values(cur, insert_query, records, **kwargs)
    conn.commit()   

def get_max_block_OHM_stake_address(cur): 
    cur.execute("SELECT max(evt_block_number) FROM OHM_stake_address;")
    return cur.fetchone()[0]
        
def get_record_count_OHM_stake_address(cur): 
    cur.execute("SELECT count(*) FROM OHM_stake_address;")
    return cur.fetchone()[0]
    
@contextmanager
def db_conn():
    conn = get_connection()
    try:
        yield conn
    finally:
        conn.close()

@contextmanager
def db_cur(connection):
    cur = connection.cursor()
    try:
        yield cur
    finally:
        cur.close()

In [9]:
def range_partition(start, end, interval): 
    """ partitions a numerical range [start, end] into a set of disjoint contiguous subranges whose union covers the input range
        example: 
            inputs: 
                start = 1000
                end = 1432
                interval = 17
            output: 
                [[1000, 1016],
                 [1017, 1033],
                     ....
                 [1408, 1424],
                 [1425, 1432]]
    """
    assert start < end 
    ss = start
    se = ss + interval - 1
    subranges = []
    while ss <= end: 
        subranges.append([ss, se])
        ss = se + 1
        se = ss + interval - 1
    if subranges: 
        subranges[-1][1] = min(subranges[-1][1], end)
    return deque(subranges)

def txs_to_OHM_staking_address_records(txs): 
    records = []
    for tx in txs: 
        if tx["tokenSymbol"] == "OHM": 
            is_from = tx["from"].lower() == OHM_STAKING_V2.lower()
            is_to = tx["to"].lower() == OHM_STAKING_V2.lower()
            if is_from or is_to: 
                v = int(tx["value"])
                if is_to: 
                    v *= -1 
                records.append((
                    tx["contractAddress"], 
                    tx["blockNumber"], 
                    dt.fromtimestamp(int(tx["timeStamp"]), tz=timezone.utc), 
                    tx["transactionIndex"],
                    tx["hash"], 
                    tx["from"].lower(), 
                    tx["to"].lower(), 
                    v
                ))
    return records

def txs_to_OHM_staking_tx_records(txs): 
    records = []
    for tx in txs: 
        if tx["tokenSymbol"] == "OHM": 
            if tx["from"].lower() == OHM_TOKEN_CONTRACT.lower() and tx["to"].lower() == OHM_STAKING_V2.lower(): 
                records.append((
                    tx["contractAddress"], 
                    tx["blockNumber"], 
                    dt.fromtimestamp(int(tx["timeStamp"]), tz=timezone.utc), 
                    tx["transactionIndex"],
                    tx["hash"], 
                    tx["from"].lower(), 
                    tx["to"].lower(), 
                    int(tx["value"])
                ))
    return records

def get_txs_for_block_range(bs, be, api_max_records): 
    # Gets all erc20 token transfer events for a smart contract between block numbers bs and be 
    # If the API doesn't return all records for the current block range, we subdivide the range 
    # in half, and recurse on both halves individually. 
    try: 
        txs = eth.get_erc20_token_transfer_events_by_address(OHM_STAKING_V2, bs, be, "asc")
    except: 
        # endpoint returns an error in case where no txs are returned for some reason lol 
        txs = []
    n = len(txs)
    if n < api_max_records: 
        return deque(txs)
    elif n == api_max_records: 
        if bs == be: 
            raise Exception(f"Block range collapsed to a single block but still got 10,000 records.")
        print(f"Block range [{bs}, {be}] returned {api_max_records} records, so dividing range in half")
        mid = bs + (be-bs) // 2
        return deque([
            *get_txs_for_block_range(bs, mid, api_max_records), 
            *get_txs_for_block_range(mid + 1, be, api_max_records)
        ])
    else: 
        raise Exception(f"Got {len(txs)} records and expected max of {api_max_records}")

class Window: 
    """ LIFO queue of fixed size """
    
    def __init__(self, window_size):
        assert window_size > 0
        self.window_size = window_size
        self.q = deque()
        
    def add(self, value): 
        self.q.append(value)
        if len(self.q) == self.window_size + 1: 
            self.q.popleft()
        
    def avg(self): 
        n = min(self.window_size, len(self.q))
        return sum(list(self.q)[-n:]) / n
    
    def reset(self): 
        self.q = deque()


run = False 
reset = False 
if run: 
    with db_conn() as conn, db_cur(conn) as cur:
        create_table_OHM_stake_address(conn, cur)
        create_table_OHM_stake_tx(conn, cur)
        # normal 
        bs, be = get_OHM_staking_block_range()
        # testing 
    #     d = dt.now().date() - datetime.timedelta(days=3)
    #     bs = 13755302
    #     be = 13761566
    #     print(d)
    #     print(bs, be)
        if reset: 
            # Start from scratch: delete existing data and query entire block range 
            rc = 0
            truncate_table_OHM_stake_address(conn, cur)
            truncate_table_OHM_stake_tx(conn, cur)
        else: 
            # Add new data to table: retain existing data and query, and determine new block start depending on db state
            rc = get_record_count_OHM_stake_address(cur)
            if rc > 0: 
                bs = get_max_block_OHM_stake_address(cur)
        print(f"Querying block range: [{bs}, {be}]")
        print(f"{be-bs} blocks in total")
        avg_blocks_per_day = math.floor(86400 / 15)
        num_blocks_per_req = avg_blocks_per_day
        block_ranges = range_partition(bs, be, num_blocks_per_req)
        # This allows us to deal with the fact that etherscan responses only return 10,000 records per request
        print(f"Requesting {avg_blocks_per_day} blocks for each single etherscan API request, roughly one day at a time.")
        i = 0
        total_attempt_insert = 0
        w = Window(5) 
        api_max_records = 10000
        backoff_thresh_f = .75
        backoff_thresh = int(backoff_thresh_f * api_max_records)
        backoff_f = .8
        while block_ranges: 
            i += 1
            bs, be = block_ranges.popleft()
            txs = get_txs_for_block_range(bs, be, api_max_records) 
            records_stake_address = txs_to_OHM_staking_address_records(txs)
            records_stake_tx = txs_to_OHM_staking_tx_records(txs)
            ntxs = len(txs)
            nr = len(records_stake_address)
            total_attempt_insert += nr
            log_props = dict(
                request=i, 
                blocks=f"[{bs}, {be}]", 
                returned=ntxs, 
                attempt_insert=nr,
                total_attempt_insert=total_attempt_insert, 
            )
            prop_strs = [f'{k}: {v}' for k,v in log_props.items()]
            print(f"-- {' - '.join(prop_strs)}")
            insert_many_table_OHM_stake_address(conn, cur, records_stake_address)
            insert_many_table_OHM_stake_tx(conn, cur, records_stake_tx)
            w.add(ntxs)
            if w.avg() > backoff_thresh: 
                # recompute remaining block ranges as rolling average number of records returned has exceeded the backoff threshold
                # this will help reduce the number of api request failures  
                w.reset() 
                num_blocks_per_req = int(num_blocks_per_req * backoff_f)
                print(f"Recomputing block ranges as we exceeded backoff threshold. Now using {num_blocks_per_req} blocks per request")
                block_ranges = range_partition(block_ranges[0][0], block_ranges[-1][1], num_blocks_per_req)
        new_rc = get_record_count_OHM_stake_address(cur)
        print(f"Finished - Before: {rc} - Attempted Insert: {total_attempt_insert} - Inserted: {new_rc-rc} - Total: {new_rc}")    

In [11]:
# fields = """encode(contract_address::bytea, 'escape') as contract_addr,
# evt_block_number, 
# evt_block_time AT TIME ZONE 'UTC', 
# evt_index, 
# encode(evt_tx_hash::bytea, 'escape') as tx_hash,
# encode(from_addr::bytea, 'escape') as from,
# encode(to_addr::bytea, 'escape') as to, 
# value 
# from OHM_stake_address""".replace("\n", '')
# conn = psycopg2.connect("dbname=olympus user=postgres password=a")
# cur = conn.cursor()
# cur.execute(
#     f"""SELECT {fields} FROM OHM_stake_address ORDER BY evt_block_time ASC"""
# )
# res = cur.fetchall()
# conn.commit()
# cur.close()
# conn.close()
# [list(r) for r in res[:1] + res]

In [12]:
# with db_conn() as conn, db_cur(conn) as cur:
#     # https://docs.olympusdao.finance/main/references/equations
#     apy_over_time_sql = """
#     with final_staked as
#     (
#         SELECT evt_block_time as second,
#                 sum(-sum(value)) over (order by evt_block_time)/1e9 as OHM_staked_amount
#         FROM OHM_stake_address
#         GROUP BY 1
#     ),

#     staking_tx as (
#         select 	evt_block_time, 
#                 (value/1e9) as ohm_transferred, 
#                 evt_tx_hash, 
#                 evt_block_number
#         FROM OHM_stake_tx
#     ),

#     rebase as (
#         select 	evt_block_time, 
#                 ohm_transferred, 
#                 ohm_staked_amount, 
#                 (ohm_transferred/(ohm_staked_amount)) as rebase, 
#                 evt_tx_hash, 
#                 evt_block_number
#         from staking_tx
#         left join final_staked on final_staked."second" = evt_block_time
#         order by evt_block_time desc
#     )

#     select 	evt_block_time, 
#             rebase*100, 
#             (1+rebase)^(1095) as apy, 
#             evt_block_number 
#     from rebase
#     where evt_block_time > '2021-06-16'
#     order by evt_block_time asc
#     """
#     cur.execute(apy_over_time_sql)
#     res = cur.fetchall()

In [13]:
# import altair as alt 
# df = pd.DataFrame(res, columns=["date", "rebase", "apy", "block_no"])
# df["date"] = pd.to_datetime(df["date"], utc=True)
# df["rebase"] = df["rebase"].astype(float)
# df["apy"] = df["apy"].astype(float)
# df["block_no"] = df["block_no"].astype(int)
# print(len(df))
# alt.Chart(df[["date", "rebase"]]).mark_line().encode(
#     alt.X('date:T', title='date'),
#     alt.Y('rebase:Q')
# ).properties(
#     height=400,
#     width=1250
# )

In [15]:
from collections import defaultdict

def point_series_to_balance_series(ts, dmin, dmax): 
    ts = [{"date": dmin, "value": 0}] + ts
    i = 0
    while i < len(ts): 
        v = ts[i]["value"]
        d = ts[i+1]["date"] if i+1 < len(ts) else dmax
        ts.insert(i+1, {"date": d, "value": v})
        i += 2
    return ts
    
def get_ohm_token_balance_for_address(addr, dmin, dmax, rebases, bs=0, be=99999999, sort='asc', log=True): 
    ohm_tokens = ["OHM", "sOHM", "gOHM", "wsOHM"]
    vmul = dict(OHM=1e-9, sOHM=1e-9, gOHM=1e-18, wsOHM=1e-18)
    res = eth.get_erc20_token_transfer_events_by_address(addr, bs, be, sort)
    # create a time series of transfer events for each OHM token 
    tsmap = defaultdict(list)
    balance = defaultdict(float)
    for tx in res: 
        symbol = tx['tokenSymbol']
        if symbol in ohm_tokens:
            sign = -1 if tx["from"] == addr else 1 
            tsmap[symbol].append({
                "date": dt.fromtimestamp(int(tx["timeStamp"]), tz=timezone.utc), 
                "value": sign * float(tx['value']) * vmul[symbol], 
                "block_no": tx['blockNumber']
            }) 
    # add transfer events for sOHM related to rebases 
    sohm_balance = 0
    sohm_transfers = deque(tsmap["sOHM"])
    rebases = deque(rebases.sort_values("date").to_dict("records")) 
    while sohm_transfers or rebases:
        if sohm_transfers and not rebases: 
            # only transfers remain
            transfer = sohm_transfers.popleft()
            if log: 
                print(f"{transfer['date']} block {transfer['block_no']} Transfer: {sohm_balance} -> {sohm_balance + transfer['value']}")
            sohm_balance += transfer["value"]
        elif not sohm_transfers and rebases:
            # only rebases remain
            rebase = rebases.popleft() 
            if log: 
                rstr = f"{(rebase['rebase'] / 100):.7f}"
                print(f"{rebase['date']} block {rebase['block_no']} Rebase ({rstr}): {sohm_balance} -> {sohm_balance * (1 + rebase['rebase'] / 100)}")
            sohm_balance *= (1 + rebase['rebase'] / 100)
        else: 
            # both transfer and rebase event 
            if sohm_transfers[0]["date"] <= rebases[0]["date"]: 
                # transfer occurred first 
                transfer = sohm_transfers.popleft()
                if log: 
                    print(f"{transfer['date']} block {transfer['block_no']} Transfer: {sohm_balance} -> {sohm_balance + transfer['value']}")
                sohm_balance += transfer["value"]
            else: 
                # rebase occurred first
                rebase = rebases.popleft() 
                if log: 
                    rstr = f"{(rebase['rebase'] / 100):.7f}"
                    print(f"{rebase['date']} block {rebase['block_no']} Rebase ({rstr}): {sohm_balance} -> {sohm_balance * (1 + rebase['rebase'] / 100)}")
                sohm_balance *= (1 + rebase['rebase'] / 100)
    balance = sohm_balance
    # convert each time series into a balance time series
    tsmap = {k: point_series_to_balance_series(v, dmin, dmax) for k, v in tsmap.items()}
    return balance, tsmap

import altair as alt 
# addr = '0x93a98e245c71dbcace51b22904181f8779306576'
addr = '0xe249d1bE97f4A716CDE0D7C5B6b682F491621C41' # mine 
staking_deploy_tx = w3.eth.get_transaction(OHM_STAKING_V2_DEPLOYMENT_TX)
staking_deploy_block = int(staking_deploy_tx["blockNumber"])
dmin = dt.fromtimestamp(int(eth.get_block_reward_by_block_number(staking_deploy_block)['timeStamp']), tz=timezone.utc)
dmax = dt.now(timezone.utc)
rebases = df[['date', 'rebase', 'block_no']]
balance, tsmap = get_ohm_token_balance_for_address(addr, dmin, dmax, rebases, log=True)

2021-06-16 04:34:57+00:00 block 12643427 Rebase (0.0071686): 0 -> 0.0
2021-06-16 13:19:02+00:00 block 12645715 Rebase (0.0073179): 0.0 -> 0.0
2021-06-16 22:02:03+00:00 block 12648046 Rebase (0.0072756): 0.0 -> 0.0
2021-06-17 10:09:51+00:00 block 12651328 Rebase (0.0069626): 0.0 -> 0.0
2021-06-17 13:53:15+00:00 block 12652328 Rebase (0.0069431): 0.0 -> 0.0
2021-06-18 00:59:46+00:00 block 12655323 Rebase (0.0066846): 0.0 -> 0.0
2021-06-18 05:56:31+00:00 block 12656726 Rebase (0.0066298): 0.0 -> 0.0
2021-06-18 14:53:03+00:00 block 12659034 Rebase (0.0065968): 0.0 -> 0.0
2021-06-18 22:33:35+00:00 block 12661109 Rebase (0.0064184): 0.0 -> 0.0
2021-06-19 07:22:49+00:00 block 12663465 Rebase (0.0063949): 0.0 -> 0.0
2021-06-19 18:33:44+00:00 block 12666447 Rebase (0.0063048): 0.0 -> 0.0
2021-06-20 03:34:22+00:00 block 12668884 Rebase (0.0062473): 0.0 -> 0.0
2021-06-20 08:30:11+00:00 block 12670176 Rebase (0.0061917): 0.0 -> 0.0
2021-06-20 16:39:23+00:00 block 12672303 Rebase (0.0061429): 0.0 -

In [105]:
# TODO: Only staking + unstaking actions are available on etherscan. Figure out how to find rebase rewards on chain 
pprint.pprint(balance)

-0.3297861948707431


In [67]:
sym_color = {"OHM": "#E4572E", "gOHM": "#29335C", "sOHM": "#A8C686", "wsOHM": "#F3A712"}
charts = []
for symbol, ts in tsmap.items(): 
    df = pd.DataFrame(ts)
    df["date"] = pd.to_datetime(df["date"], utc=True)#.dt.tz_localize(None)
#     df["value"] = df["value"].apply(lambda v: round(v, 7))
    c = alt.Chart(df).mark_line(color=sym_color[symbol]).encode(
        alt.X("date:T"), 
        alt.Y("value:Q", title=symbol), 
    ).properties(height=200, width=1000)
    charts.append(c)
    
alt.vconcat(*charts)

alt.VConcatChart(...)

In [53]:
# for sym, ts in tsmap.items():
#     print(sym, round(ts[-1]['value'], 6))

OHM 0.0
sOHM -0.0
wsOHM 0.0
gOHM 5.674899


In [120]:
# https://etherscan.io/tokencheck-tool
OHM_TOKEN_CONTRACT_ADDRS = dict(
    sOHM='0x04f2694c8fcee23e8fd0dfea1d4f5bb8c352111f', 
    gOHM='0x0ab87046fBb341D058F17CBC4c1133F25a20a52f'
)
float(eth.get_acc_balance_by_token_and_contract_address(OHM_TOKEN_CONTRACT_ADDRS['sOHM'], addr)) * 10**-9

6.494237221000001

In [70]:
# url = "https://api.etherscan.io/api?module=logs"
# kwargs = dict(
#     action='getLogs', 
#     fromBlock=0, 
#     toBlock=99999999, 
#     address='0xc58e923bf8a00e4361fe3f4275226a543d7d3ce6', 
#     topic0='0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0', 
#     apikey=API_KEY
# )
# url += ''.join([f"&{k}={v}" for k, v in kwargs.items()])
# print(url)
# res = requests.get(url).json()["result"]
# for r in res: 
#     print(r)

https://api.etherscan.io/api?module=logs&action=getLogs&fromBlock=0&toBlock=99999999&address=0xc58e923bf8a00e4361fe3f4275226a543d7d3ce6&topic0=0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0&apikey=VR7YA9DRDB4Y15B5N3WU9E7PSJ9RWPCP5S
{'address': '0xc58e923bf8a00e4361fe3f4275226a543d7d3ce6', 'topics': ['0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0', '0x0000000000000000000000000000000000000000000000000000000000000000', '0x0000000000000000000000003524c03d39a13d51485419a17586286a6b617dd3'], 'data': '0x', 'blockNumber': '0xc165db', 'timeStamp': '0x60cfdee3', 'gasPrice': '0x2540be400', 'gasUsed': '0x1553fe', 'logIndex': '0x9e', 'transactionHash': '0x193ae125fe3de6cc9c799a7d62ac446eb744966e0d0ac77d5a80eea25b099ef1', 'transactionIndex': '0x65'}
{'address': '0xc58e923bf8a00e4361fe3f4275226a543d7d3ce6', 'topics': ['0x8be0079c531659141344cd1fd0a4f28419497f9722a3daafe3b4186f6b6457e0', '0x0000000000000000000000003524c03d39a13d51485419a17586286a6b617dd3', '0x0

In [147]:
f"""
https://api.etherscan.io/api
   ?module=account
   &action=txlistinternal
   &address={'0x2c1ba59d6f58433fb1eaee7d20b26ed83bda51a3'}
   &startblock=0
   &endblock=99999999
   &sort=asc
   &apikey={API_KEY}
""".replace(" ", '').replace('\n', '')

'https://api.etherscan.io/api?module=account&action=txlistinternal&address=0x2c1ba59d6f58433fb1eaee7d20b26ed83bda51a3&startblock=0&endblock=99999999&sort=asc&apikey=VR7YA9DRDB4Y15B5N3WU9E7PSJ9RWPCP5S'

In [181]:
import time 
import random 

headers={'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
url_base = f'https://etherscan.io/txsInternal?a={OHM_STAKING_DISTRIBUTOR_V4}&&m=advanced&p='
i = 1
block_tx_map = dict()
while True: 
    url = url_base + str(i)
    # fake a User-Agent as this bypasses captcha requirements 
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.content, 'html.parser')
    tables = [
        t for t in soup.find_all('table') 
        if "Txn Hash" in str(t) and re.sub("[\s\n\r]", "", t.find("tbody").getText())
    ]
    assert len(tables) == 1
    df = pd.read_html(tables[0].prettify())[0]
    print(f"Page: {i} - Record Count: {len(df)}")
    if len(df) > 1:
        df['Block'] = df['Block'].fillna(-1).astype(int)
        for row in df.to_dict(orient="records"): 
            if row['Block'] != -1 and row["Parent Txn Hash"]: 
                block_tx_map[row['Block']] = row["Parent Txn Hash"]
    else: 
        break          
    i += 1
    time.sleep(3 * random.random() + 3 * random.random() + 3)

Page: 1 - Record Count: 50
Page: 2 - Record Count: 50
Page: 3 - Record Count: 50
Page: 4 - Record Count: 50
Page: 5 - Record Count: 50
Page: 6 - Record Count: 50
Page: 7 - Record Count: 50
Page: 8 - Record Count: 50
Page: 9 - Record Count: 50
Page: 10 - Record Count: 50
Page: 11 - Record Count: 50
Page: 12 - Record Count: 50
Page: 13 - Record Count: 50
Page: 14 - Record Count: 50
Page: 15 - Record Count: 50
Page: 16 - Record Count: 50
Page: 17 - Record Count: 50
Page: 18 - Record Count: 50
Page: 19 - Record Count: 50
Page: 20 - Record Count: 50
Page: 21 - Record Count: 50
Page: 22 - Record Count: 50
Page: 23 - Record Count: 50
Page: 24 - Record Count: 50
Page: 25 - Record Count: 50
Page: 26 - Record Count: 50
Page: 27 - Record Count: 50
Page: 28 - Record Count: 50
Page: 29 - Record Count: 50
Page: 30 - Record Count: 50
Page: 31 - Record Count: 50
Page: 32 - Record Count: 50
Page: 33 - Record Count: 50
Page: 34 - Record Count: 50
Page: 35 - Record Count: 50
Page: 36 - Record Count: 50
P

In [236]:
def get_tx_logs_for_staking_distributor(txhash): 
    # staking distributor etherscan internal txs: https://etherscan.io/address/0xc58e923bf8a00e4361fe3f4275226a543d7d3ce6/advanced#internaltx
    # https://docs.olympusdao.finance/main/basics/basics#how-do-i-track-my-rebase-rewards
    # for each parent tx of the internal txs found in the above link, 
#     print(txhash)
    contract_sohm = w3.eth.contract(
        address='0x04F2694C8fcee23e8Fd0dfEA1d4f5Bb8c352111F', 
        abi='[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"owner","type":"address"},{"indexed":true,"internalType":"address","name":"spender","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Approval","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"epoch","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"rebase","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"index","type":"uint256"}],"name":"LogRebase","type":"event"},{"anonymous":false,"inputs":[{"indexed":false,"internalType":"address","name":"stakingContract","type":"address"}],"name":"LogStakingContractUpdated","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"uint256","name":"epoch","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"timestamp","type":"uint256"},{"indexed":false,"internalType":"uint256","name":"totalSupply","type":"uint256"}],"name":"LogSupply","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipPulled","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"previousOwner","type":"address"},{"indexed":true,"internalType":"address","name":"newOwner","type":"address"}],"name":"OwnershipPushed","type":"event"},{"anonymous":false,"inputs":[{"indexed":true,"internalType":"address","name":"from","type":"address"},{"indexed":true,"internalType":"address","name":"to","type":"address"},{"indexed":false,"internalType":"uint256","name":"value","type":"uint256"}],"name":"Transfer","type":"event"},{"inputs":[],"name":"DOMAIN_SEPARATOR","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"INDEX","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"PERMIT_TYPEHASH","outputs":[{"internalType":"bytes32","name":"","type":"bytes32"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner_","type":"address"},{"internalType":"address","name":"spender","type":"address"}],"name":"allowance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"approve","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"gons","type":"uint256"}],"name":"balanceForGons","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"who","type":"address"}],"name":"balanceOf","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"circulatingSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"decimals","outputs":[{"internalType":"uint8","name":"","type":"uint8"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"subtractedValue","type":"uint256"}],"name":"decreaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"amount","type":"uint256"}],"name":"gonsForBalance","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"addedValue","type":"uint256"}],"name":"increaseAllowance","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"index","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"stakingContract_","type":"address"}],"name":"initialize","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"initializer","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"manager","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"name","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"}],"name":"nonces","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"owner","type":"address"},{"internalType":"address","name":"spender","type":"address"},{"internalType":"uint256","name":"amount","type":"uint256"},{"internalType":"uint256","name":"deadline","type":"uint256"},{"internalType":"uint8","name":"v","type":"uint8"},{"internalType":"bytes32","name":"r","type":"bytes32"},{"internalType":"bytes32","name":"s","type":"bytes32"}],"name":"permit","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"pullManagement","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"newOwner_","type":"address"}],"name":"pushManagement","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"profit_","type":"uint256"},{"internalType":"uint256","name":"epoch_","type":"uint256"}],"name":"rebase","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"","type":"uint256"}],"name":"rebases","outputs":[{"internalType":"uint256","name":"epoch","type":"uint256"},{"internalType":"uint256","name":"rebase","type":"uint256"},{"internalType":"uint256","name":"totalStakedBefore","type":"uint256"},{"internalType":"uint256","name":"totalStakedAfter","type":"uint256"},{"internalType":"uint256","name":"amountRebased","type":"uint256"},{"internalType":"uint256","name":"index","type":"uint256"},{"internalType":"uint256","name":"blockNumberOccured","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"renounceManagement","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"uint256","name":"_INDEX","type":"uint256"}],"name":"setIndex","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"stakingContract","outputs":[{"internalType":"address","name":"","type":"address"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"symbol","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"totalSupply","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transfer","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"},{"inputs":[{"internalType":"address","name":"from","type":"address"},{"internalType":"address","name":"to","type":"address"},{"internalType":"uint256","name":"value","type":"uint256"}],"name":"transferFrom","outputs":[{"internalType":"bool","name":"","type":"bool"}],"stateMutability":"nonpayable","type":"function"}]'
    )
    receipt = w3.eth.get_transaction_receipt(txhash)
#     print(receipt)
    if receipt.status == 0: 
        print("Skipping failed transaction")
        return None 
    logs_rebase = contract_sohm.events.LogRebase().processReceipt(receipt)
    logs_supply = contract_sohm.events.LogSupply().processReceipt(receipt)
    if not len(logs_rebase) == len(logs_supply) == 1: 
        print("Skipping transaction that doesn't have logSupply and logRebase events")
        return None 
    logs_rebase = logs_rebase[0]
    logs_supply = logs_supply[0]
    assert logs_rebase.args.epoch == logs_supply.args.epoch
    epoch = logs_rebase.args.epoch
    rebase = logs_rebase.args.rebase
    index = logs_rebase.args.index 
    totalSupply = logs_supply.args.totalSupply
    return epoch, rebase, index, totalSupply 

block_data = dict()
for block, txhash in block_tx_map.items(): 
    d = dt.fromtimestamp(w3.eth.get_block(block).timestamp, tz=timezone.utc)
    res = get_tx_logs_for_staking_distributor(txhash)
    if res: 
        epoch, rebase, index, totalSupply = res
#         print(f"epoch: {epoch} - rebase: {rebase} - index: {index} - totalSupply: {totalSupply}")
        block_data[block] = (d, epoch, rebase, index, totalSupply)
    
# pprint.pprint(block_tx_map)

/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xdbdd768aaeaa05b9f725a2d7926bf294a4fdfc02da0d85afc4d35622f1fc1966') and logIndex: 23 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xdbdd768aaeaa05b9f725a2d7926bf294a4fdfc02da0d85afc4d35622f1fc1966') and logIndex: 25 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xdbdd768aaeaa05b9f725a2d7926bf294a4fdfc02da0d85afc4d35622f1fc1966') and logI

Skipping failed transaction
Skipping failed transaction


/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xf20a0ff92a7515515f45c226df66687f11fc2ac86ee8ba703aa53e4a2449cab4') and logIndex: 254 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xf20a0ff92a7515515f45c226df66687f11fc2ac86ee8ba703aa53e4a2449cab4') and logIndex: 256 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xf20a0ff92a7515515f45c226df66687f11fc2ac86ee8ba703aa53e4a2449cab4') and lo

Skipping transaction that doesn't have logSupply and logRebase events


/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x72bbe73d428733c3253b79fa92aea399f997053b7324bff313244ce1872689b0') and logIndex: 160 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x72bbe73d428733c3253b79fa92aea399f997053b7324bff313244ce1872689b0') and logIndex: 162 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x72bbe73d428733c3253b79fa92aea399f997053b7324bff313244ce1872689b0') and lo

...

/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x2d6a483073125d67709b052f8ac50c6f4abdf08519a7d7686d0acac86632c660') and logIndex: 141 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x2d6a483073125d67709b052f8ac50c6f4abdf08519a7d7686d0acac86632c660') and logIndex: 143 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x2d6a483073125d67709b052f8ac50c6f4abdf08519a7d7686d0acac86632c660') and lo

Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events


/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x6279cce8e0973df33f622995b3d5f86c16d19eb787514e6f834e6c9fdfe7cf35') and logIndex: 328 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x6279cce8e0973df33f622995b3d5f86c16d19eb787514e6f834e6c9fdfe7cf35') and logIndex: 329 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0x6279cce8e0973df33f622995b3d5f86c16d19eb787514e6f834e6c9fdfe7cf35') and lo

Skipping transaction that doesn't have logSupply and logRebase events
Skipping transaction that doesn't have logSupply and logRebase events


/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xf5a03a9d7b0c9c8acb25782e485e130736aab2b7125cc5b6f9ed545cb2c43c07') and logIndex: 390 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xf5a03a9d7b0c9c8acb25782e485e130736aab2b7125cc5b6f9ed545cb2c43c07') and logIndex: 391 encountered the following error during processing: MismatchedABI(The event signature did not match the provided ABI). It has been discarded.
  warnings.warn(
/Users/ALEX/opt/anaconda3/envs/finances/lib/python3.9/site-packages/web3/contract.py:1166: UserWarning: The log with transaction hash: HexBytes('0xf5a03a9d7b0c9c8acb25782e485e130736aab2b7125cc5b6f9ed545cb2c43c07') and lo

In [246]:
ts = []
items = [(block, *values) for block, values in reversed(block_data.items())]
for i in range(1, len(items)): 
    ts.append({
        "block_no": items[i][0], 
        "date": items[i][1], 
        "rebase": ((items[i][4] / items[i-1][4]) - 1) * 100}
    )
rebases = ts

In [247]:
# Now that we have the index over time, we can use this to track the growth of holdings over time 
def get_ohm_token_balance_for_address(addr, dmin, dmax, rebases, bs=0, be=99999999, sort='asc', log=True): 
    ohm_tokens = ["OHM", "sOHM", "gOHM", "wsOHM"]
    vmul = dict(OHM=1e-9, sOHM=1e-9, gOHM=1e-18, wsOHM=1e-18)
    res = eth.get_erc20_token_transfer_events_by_address(addr, bs, be, sort)
    # create a time series of transfer events for each OHM token 
    tsmap = defaultdict(list)
    balance = defaultdict(float)
    for tx in res: 
        symbol = tx['tokenSymbol']
        if symbol in ohm_tokens:
            sign = -1 if tx["from"] == addr else 1 
            tsmap[symbol].append({
                "date": dt.fromtimestamp(int(tx["timeStamp"]), tz=timezone.utc), 
                "value": sign * float(tx['value']) * vmul[symbol], 
                "block_no": tx['blockNumber']
            }) 
    # add transfer events for sOHM related to rebases 
    sohm_balance = 0
    sohm_transfers = deque(tsmap["sOHM"])
    rebases = deque(rebases) 
    while sohm_transfers or rebases:
        if sohm_transfers and not rebases: 
            # only transfers remain
            transfer = sohm_transfers.popleft()
            if log: 
                print(f"{transfer['date']} block {transfer['block_no']} Transfer: {sohm_balance} -> {sohm_balance + transfer['value']}")
            sohm_balance += transfer["value"]
        elif not sohm_transfers and rebases:
            # only rebases remain
            rebase = rebases.popleft() 
            if log: 
                rstr = f"{(rebase['rebase'] / 100):.7f}"
                print(f"{rebase['date']} block {rebase['block_no']} Rebase ({rstr}): {sohm_balance} -> {sohm_balance * (1 + rebase['rebase'] / 100)}")
            sohm_balance *= (1 + rebase['rebase'] / 100)
        else: 
            # both transfer and rebase event 
            if sohm_transfers[0]["date"] <= rebases[0]["date"]: 
                # transfer occurred first 
                transfer = sohm_transfers.popleft()
                if log: 
                    print(f"{transfer['date']} block {transfer['block_no']} Transfer: {sohm_balance} -> {sohm_balance + transfer['value']}")
                sohm_balance += transfer["value"]
            else: 
                # rebase occurred first
                rebase = rebases.popleft() 
                if log: 
                    rstr = f"{(rebase['rebase'] / 100):.7f}"
                    print(f"{rebase['date']} block {rebase['block_no']} Rebase ({rstr}): {sohm_balance} -> {sohm_balance * (1 + rebase['rebase'] / 100)}")
                sohm_balance *= (1 + rebase['rebase'] / 100)
    balance = sohm_balance
    # convert each time series into a balance time series
    tsmap = {k: point_series_to_balance_series(v, dmin, dmax) for k, v in tsmap.items()}
    return balance, tsmap

import altair as alt 
# addr = '0x93a98e245c71dbcace51b22904181f8779306576'
addr = '0xe249d1bE97f4A716CDE0D7C5B6b682F491621C41' # mine 
staking_deploy_tx = w3.eth.get_transaction(OHM_STAKING_V2_DEPLOYMENT_TX)
staking_deploy_block = int(staking_deploy_tx["blockNumber"])
dmin = dt.fromtimestamp(int(eth.get_block_reward_by_block_number(staking_deploy_block)['timeStamp']), tz=timezone.utc)
dmax = dt.now(timezone.utc)
balance, tsmap = get_ohm_token_balance_for_address(addr, dmin, dmax, rebases, log=True)

2021-06-23 17:55:33+00:00 block 12691856 Rebase (0.0117641): 0 -> 0.0
2021-06-24 01:42:45+00:00 block 12694003 Rebase (0.0058498): 0.0 -> 0.0
2021-06-24 10:08:59+00:00 block 12696223 Rebase (0.0058429): 0.0 -> 0.0
2021-06-24 18:22:43+00:00 block 12698405 Rebase (0.0057589): 0.0 -> 0.0
2021-06-25 02:23:26+00:00 block 12700613 Rebase (0.0057750): 0.0 -> 0.0
2021-06-25 10:49:01+00:00 block 12702813 Rebase (0.0057105): 0.0 -> 0.0
2021-06-25 19:13:48+00:00 block 12705021 Rebase (0.0056958): 0.0 -> 0.0
2021-06-26 03:23:37+00:00 block 12707224 Rebase (0.0056094): 0.0 -> 0.0
2021-06-26 11:24:16+00:00 block 12709404 Rebase (0.0055976): 0.0 -> 0.0
2021-06-26 19:52:25+00:00 block 12711607 Rebase (0.0055472): 0.0 -> 0.0
2021-06-27 03:55:44+00:00 block 12713809 Rebase (0.0055261): 0.0 -> 0.0
2021-06-27 12:18:15+00:00 block 12716013 Rebase (0.0054969): 0.0 -> 0.0
2021-06-27 20:19:06+00:00 block 12718208 Rebase (0.0054827): 0.0 -> 0.0
2021-06-28 04:40:19+00:00 block 12720433 Rebase (0.0054415): 0.0 -

In [ ]:
OHM_D